# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 308 kB 5.5 MB/s 
     |████████████████████████████████| 80 kB 6.5 MB/s 
     |████████████████████████████████| 209 kB 48.4 MB/s 
     |████████████████████████████████| 75 kB 2.9 MB/s 
     |████████████████████████████████| 49 kB 3.3 MB/s 
     |████████████████████████████████| 111 kB 37.8 MB/s 
     |████████████████████████████████| 144 kB 37.0 MB/s 


In [2]:
import optuna

optuna.__version__

'2.10.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9533333333333333


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-10-10 15:07:34,250] A new study created in memory with name: no-name-feadaeca-5265-4c3e-bfcb-a8ff573e2946
[I 2021-10-10 15:07:35,499] Trial 0 finished with value: 0.96 and parameters: {'n_estimators': 19, 'max_depth': 1.1119769847073764}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:07:35,590] Trial 1 finished with value: 0.96 and parameters: {'n_estimators': 12, 'max_depth': 22.562901558799375}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:07:35,622] Trial 2 finished with value: 0.9333333333333332 and parameters: {'n_estimators': 2, 'max_depth': 24.22632248839123}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:07:35,699] Trial 3 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 11, 'max_depth': 10.39068057426003}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:07:35,759] Trial 4 finished with value: 0.8533333333333334 and parameters: {'n_estimators': 9, 'max_depth': 1.0296925451855585}. Best is trial 0 with value: 0.96.
[I 2021-10-10

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 5, 'max_depth': 14.497811531365288}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-10-10 15:12:47,150] A new study created in memory with name: no-name-298638cc-4cb3-4ae3-a021-87ccb9a75f06
[I 2021-10-10 15:12:48,116] Trial 0 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 134.8572136371431}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:12:48,148] Trial 1 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 2550490167.085934}. Best is trial 0 with value: 0.96.
[I 2021-10-10 15:12:48,171] Trial 2 finished with value: 0.9733333333333333 and parameters: {'classifier': 'SVC', 'svc_c': 0.9868829239262082}. Best is trial 2 with value: 0.9733333333333333.
[I 2021-10-10 15:12:48,191] Trial 3 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 1.9918124372157155e-09}. Best is trial 2 with value: 0.9733333333333333.
[I 2021-10-10 15:12:48,279] Trial 4 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 7, 'max_depth': 15.47457983139648}. Best is trial 2 with valu

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 3.7755978290060717}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])